# Problem Statement-
### It happens all the time: someone gives you data containing malformed strings, Python, lists and missing data. How do you tidy it up so you can get on with the analysis?

In [465]:
import numpy as np
import pandas as pd

In [466]:
df = pd.DataFrame({'From_To': ['LoNDon_paris', 'MAdrid_miLAN',
'londON_StockhOlm', 'Budapest_PaRis', 'Brussels_londOn'],
'FlightNumber': [10045, np.nan, 10065, np.nan, 10085],
'RecentDelays': [[23, 47], [], [24, 43, 87], [13], [67, 32]],
'Airline': ['KLM(!)', '<Air France> (12)', '(British Airways. )', '12. Air France', '"Swiss Air"']})

In [467]:
df

,From_To,FlightNumber,RecentDelays,Airline
0,LoNDon_paris,10045.0,"[23, 47]",KLM(!)
1,MAdrid_miLAN,NaN,[],<Air France> (12)
2,londON_StockhOlm,10065.0,"[24, 43, 87]",(British Airways. )
3,Budapest_PaRis,NaN,[13],12. Air France
4,Brussels_londOn,10085.0,"[67, 32]","""Swiss Air"""


In [468]:
df.dtypes

From_To          object
FlightNumber    float64
RecentDelays     object
Airline          object
dtype: object

### 1. Some values in the the FlightNumber column are missing. These numbers are meant to increase by 10 with each row so 10055 and 10075 need to be put in place. Fill in these missing numbers and make the column an integer column (instead of a float column).

In [470]:
#assigning values at missing locations
df.iloc[1,1]='10055'
df.iloc[3,1]='10075'

In [471]:
df

,From_To,FlightNumber,RecentDelays,Airline
0,LoNDon_paris,10045,"[23, 47]",KLM(!)
1,MAdrid_miLAN,10055,[],<Air France> (12)
2,londON_StockhOlm,10065,"[24, 43, 87]",(British Airways. )
3,Budapest_PaRis,10075,[13],12. Air France
4,Brussels_londOn,10085,"[67, 32]","""Swiss Air"""


In [472]:
df[['FlightNumber']] = df[['FlightNumber']].astype(int)
df

,From_To,FlightNumber,RecentDelays,Airline
0,LoNDon_paris,10045,"[23, 47]",KLM(!)
1,MAdrid_miLAN,10055,[],<Air France> (12)
2,londON_StockhOlm,10065,"[24, 43, 87]",(British Airways. )
3,Budapest_PaRis,10075,[13],12. Air France
4,Brussels_londOn,10085,"[67, 32]","""Swiss Air"""


In [473]:
df.dtypes

From_To         object
FlightNumber     int32
RecentDelays    object
Airline         object
dtype: object

### 2. The From_To column would be better as two separate columns! Split each string on the underscore delimiter _ to give a new temporary DataFrame with the correct values. Assign the correct column names to this temporary DataFrame.

In [474]:
#creating temporary dataframe which has two columns 'From' and 'To'
tmpDF = pd.DataFrame(columns=['From','To'])

#splitting column with underscore delimiter
tmpDF[['From','To']] = df.From_To.str.split('_',expand=True)
tmpDF

,From,To
0,LoNDon,paris
1,MAdrid,miLAN
2,londON,StockhOlm
3,Budapest,PaRis
4,Brussels,londOn


In [475]:
tmpDF.dtypes

From    object
To      object
dtype: object

### 3. Notice how the capitalisation of the city names is all mixed up in this temporary DataFrame. Standardise the strings so that only the first letter is uppercase (e.g. "londON" should become "London".)

In [476]:
df1=tmpDF['From'].apply(lambda x: x.capitalize())
df1

0      London
1      Madrid
2      London
3    Budapest
4    Brussels
Name: From, dtype: object

In [477]:
#Converting first letter of column 'From' to uppercase
tmpDF['From'] = tmpDF['From'].str.capitalize()
tmpDF

,From,To
0,London,paris
1,Madrid,miLAN
2,London,StockhOlm
3,Budapest,PaRis
4,Brussels,londOn


In [478]:
#Converting first letter of column 'To' to uppercase
tmpDF['To'] = tmpDF['To'].str.capitalize()
tmpDF

,From,To
0,London,Paris
1,Madrid,Milan
2,London,Stockholm
3,Budapest,Paris
4,Brussels,London


### 4. Delete the From_To column from df and attach the temporary DataFrame from the previous questions.

In [479]:
#Deleting column from the dataframe
df=df.drop(columns="From_To")
df

,FlightNumber,RecentDelays,Airline
0,10045,"[23, 47]",KLM(!)
1,10055,[],<Air France> (12)
2,10065,"[24, 43, 87]",(British Airways. )
3,10075,[13],12. Air France
4,10085,"[67, 32]","""Swiss Air"""


In [480]:
#concatenating temporary dataframe and existing dataframe
result = pd.concat([tmpDF, df], axis=1, sort=False)

In [481]:
result

,From,To,FlightNumber,RecentDelays,Airline
0,London,Paris,10045,"[23, 47]",KLM(!)
1,Madrid,Milan,10055,[],<Air France> (12)
2,London,Stockholm,10065,"[24, 43, 87]",(British Airways. )
3,Budapest,Paris,10075,[13],12. Air France
4,Brussels,London,10085,"[67, 32]","""Swiss Air"""


### 5. In the RecentDelays column, the values have been entered into the DataFrame as a list. We would like each first value in its own column, each second value in its own column, and so on. If there isn't an Nth value, the value should be NaN.
### Expand the Series of lists into a DataFrame named delays, rename the columns delay_1, delay_2, etc. and replace the unwanted RecentDelays column in df with delays.

In [482]:
result[['RecentDelays']].dtypes

RecentDelays    object
dtype: object

In [483]:
type(result[['RecentDelays']].dtypes)

pandas.core.series.Series

In [484]:
result['RecentDelays']

0        [23, 47]
1              []
2    [24, 43, 87]
3            [13]
4        [67, 32]
Name: RecentDelays, dtype: object

In [485]:
# expand into its own dataframe
RecentDelays = result['RecentDelays'].apply(pd.Series)

# rename each variable is RecentDealys
RecentDelays = RecentDelays.rename(columns = lambda x : 'delay_' + str(x))

RecentDelays

,delay_0,delay_1,delay_2
0,23.0,47.0,NaN
1,NaN,NaN,NaN
2,24.0,43.0,87.0
3,13.0,NaN,NaN
4,67.0,32.0,NaN


In [486]:
#deleting column from dataframe
result=result.drop(columns="RecentDelays")
result

,From,To,FlightNumber,Airline
0,London,Paris,10045,KLM(!)
1,Madrid,Milan,10055,<Air France> (12)
2,London,Stockholm,10065,(British Airways. )
3,Budapest,Paris,10075,12. Air France
4,Brussels,London,10085,"""Swiss Air"""


In [487]:
#concatenating dataframes with axis=1 (column)
result1 = pd.concat([result, RecentDelays], axis=1, sort=False)
result1

,From,To,FlightNumber,Airline,delay_0,delay_1,delay_2
0,London,Paris,10045,KLM(!),23.0,47.0,NaN
1,Madrid,Milan,10055,<Air France> (12),NaN,NaN,NaN
2,London,Stockholm,10065,(British Airways. ),24.0,43.0,87.0
3,Budapest,Paris,10075,12. Air France,13.0,NaN,NaN
4,Brussels,London,10085,"""Swiss Air""",67.0,32.0,NaN


In [488]:
#assigning column 'Airline' to temporary 'mid'
mid = result1['Airline']

#deleting column 'Airline' from existing dataframe
result1.drop(labels=['Airline'], axis=1, inplace = True)

#inserting column which is assigned to 'mid' at index 6 in existing dataframe
result1.insert(6, 'Airline', mid)
result1

# OR the other way to do this
# result1 = result1[['From','To','FlightNumber','delay_0','delay_1','delay_2','Airline']]
# result1

,From,To,FlightNumber,delay_0,delay_1,delay_2,Airline
0,London,Paris,10045,23.0,47.0,NaN,KLM(!)
1,Madrid,Milan,10055,NaN,NaN,NaN,<Air France> (12)
2,London,Stockholm,10065,24.0,43.0,87.0,(British Airways. )
3,Budapest,Paris,10075,13.0,NaN,NaN,12. Air France
4,Brussels,London,10085,67.0,32.0,NaN,"""Swiss Air"""
